In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 3.8MB 45.8MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [13]:
########## 한글 전처리부분입니다 ########

import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

naver_data = pd.read_table("./drive/My Drive/ratings.txt", encoding='utf-8')
# 단어 아니면 삭제
naver_data['document'] = naver_data['document'].str.replace("[^\w]", " ")
# 혹시 공백이 있으면 null array로
naver_data['document'] = naver_data['document'].replace('', np.nan)
naver_data['label'] = naver_data['label'].replace('', np.nan)
# null array 모두 제거 (공백인 열 모두 제거)
naver_data = naver_data.dropna(how='any')

print("# preproecssing done")

# test/train 스플릿하고
review_train, review_test, y_train, y_test = train_test_split(naver_data['document'], naver_data['label'], test_size=0.25, shuffle=False, random_state=1004)

print('# split done')

# stopwords 지정
stopwords = ['를', '을', '이', '가', '으로', '로', '에', '에서']

# 토큰화 진행
X_train = []
for stc in review_train:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in review_test:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print('# tokenization done')

# preproecssing done
# split done
# tokenization done


In [14]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# X_train 단어들을 토대로 정수 인덱스 설정, 전체 단어 갯수 설정
# 유의미한 단어? 빈도수가 1~2개인 단어 버려도 큰 영향을 끼치지 않을것 -> count함수써서 빈도수 낮은 것들을 버리고, 남은 단어의 갯수들
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(X_train)

# 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')

# int_encoding done


In [71]:
word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

# word_index 저장
import json
json = json.dumps(word_index)
f3 = open("/content/drive/My Drive/wordIndex.json", "w")
f3.write(json)
f3.close()


47172개의 고유한 토큰을 찾았습니다.


In [72]:
print(X_train[1])
print(y_train[1])

[   0    0    0    0    0    0    0    0    0 3677  489 1464 1489 7512
   52   34    6 4752  861 1395    3 1252 6343 1549  402  391  739   34
  611  506  140   62 1229 1166   20 4752   12  102   52   20   43    6
  535 1465 1328  539   63   12   29  553]
1


In [73]:
# 문장 길이를 맞춰준다
# 임의로 맞추는게 아니고, 데이터셋을 보면서 최대 문장의 길이가 얼마인지 확인하시고 거기에 맞춰서
# imdb일때는 500, 네이버일때는 50으로 맞췄었습니다!
max_len = 50
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [74]:
print(X_train[1])

[   0    0    0    0    0    0    0    0    0 3677  489 1464 1489 7512
   52   34    6 4752  861 1395    3 1252 6343 1549  402  391  739   34
  611  506  140   62 1229 1166   20 4752   12  102   52   20   43    6
  535 1465 1328  539   63   12   29  553]


In [75]:
# 레이어들을 쌓을 모델을 생성
model = Sequential()
# 단어를 임베딩하는데, 5000개 (imdb) 혹은 20000개 (네이버) 의 단어를 120차원으로 내보내겠다
# 1인자 = 내가 넣을 단어의 갯수 (총 인덱스의 갯수), 2인자 = 출력할 차원 (덴스 벡터의 차원), 3인자 = 문장의 길이
model.add(Embedding(20000, 120))
# RNN - simpleRNN / LSTM
model.add(LSTM(120))
# 긍정/부정을 판단하니까 이진 분류 -> sigmoid 함수 사용
model.add(Dense(1, activation='sigmoid'))

In [76]:
# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [77]:
# 긍정/부정을 판단하니까 손실함수는 이진 교차 엔트로피, 최적화는 adam, 평가 기준은 acc (출력할때 뜬다)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64, callbacks=[early_stop, model_check])

2344/2344 [==============================] - ETA: 0s - loss: 0.3451 - acc: 0.8464
Epoch 00001: val_acc improved from -inf to 0.77125, saving model to the_best.h5
2344/2344 [==============================] - 323s 138ms/step - loss: 0.3451 - acc: 0.8464 - val_loss: 0.5175 - val_acc: 0.7713


In [21]:
# 정확도 측정
# 출력하면 [loss, acc]
print(model.evaluate(X_test, y_test))
# X 값은 전처리, 토큰화, 정수인코딩이 된 상태의 문장이어야


1563/1563 [==============================] - 27s 17ms/step - loss: 0.5246 - acc: 0.7649
[0.5246239304542542, 0.7648506164550781]


In [78]:
model.save('/content/drive/My Drive/model_best.h5')